In [11]:
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import binom
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind


In [42]:
# Lendo o Excel
df = pd.ExcelFile('Prova ADV.xlsx')

# Lendo as planilhas
base1= pd.read_excel(df, 'BASE - Pergunta 1')


### Cálculo Roi

In [66]:
#Custo Frete
base1['Custo_frete'] =   base1['Despesa de Frete'] - base1['Receita de Frete'] 

In [67]:
base_calculo = base1.groupby(['Grupo do Teste','Estado']).sum()
base_calculo

Receita de Produto  Receita de Frete  Despesa de Frete  \
Grupo do Teste Estado                                                           
A              RJ               860092.65          21677.54          21677.54   
               SP              1386543.09          28110.11          28110.11   
B              RJ              1118669.16           6603.39          24114.46   
               SP              1597880.58          10449.54          32054.40   

                       Custo_frete  
Grupo do Teste Estado               
A              RJ             0.00  
               SP             0.00  
B              RJ         17511.07  
               SP         21604.86

In [68]:
ROI_RJ = (base_calculo.iloc[2,0] - base_calculo.iloc[2,3]) / base_calculo.iloc[2,3]
print("ROI Rio de janeiro: ", ROI_RJ)

ROI_SP = (base_calculo.iloc[3,0] - base_calculo.iloc[3,3]) / base_calculo.iloc[3,3]
print("ROI São Paulo: ",ROI_SP)

ROI Rio de janeiro:  62.88354109714577
ROI São Paulo:  72.95931193259271


### Teste T

In [45]:
base1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Grupo do Teste      3220 non-null   object 
 1   Estado              3220 non-null   object 
 2   Código Pedido       3220 non-null   object 
 3   Receita de Produto  3220 non-null   float64
 4   Receita de Frete    3220 non-null   float64
 5   Despesa de Frete    3220 non-null   float64
 6   Custo_frete         3220 non-null   float64
dtypes: float64(4), object(3)
memory usage: 176.2+ KB


In [57]:
#Separando os estados
estado_sp = base1[base1["Estado"] == "SP"]
estado_rj = base1[base1["Estado"] == "RJ"]

Cálculo das taxas de conversão para cada estado e grupo

In [73]:
tx_sp_a, tx_sp_b = conv_a_sp/populacao_sp, conv_b_sp/populacao_sp

print("Taxa SP, grupo A: ", tx_sp_a*100, "  grupo B: ",tx_sp_b*100 )

Taxa SP, grupo A:  2.124110320284698   grupo B:  2.3265124555160144


In [74]:
tx_rj_a, tx_rj_b = conv_a_rj/populacao_rj, conv_b_rj/populacao_rj

print("Taxa RJ, grupo A: ", tx_rj_a*100, "  grupo B: ",tx_rj_b*100 )

Taxa RJ, grupo A:  2.100271002710027   grupo B:  2.4879554351099067


In [76]:
#Populão do estudo em SP
populacao_sp = 44960

#Quantidade de individuos para cada grupo
conv_a_sp, conv_b_sp = estado_sp['Grupo do Teste'].value_counts()[1], estado_sp['Grupo do Teste'].value_counts()[0]

In [77]:
#Populão do estudo no RJ
populacao_rj = 26568

#Quantidade de individuos para cada grupo
conv_a_rj, conv_b_rj = estado_rj['Grupo do Teste'].value_counts()[1], estado_rj['Grupo do Teste'].value_counts()[0]

In [79]:
#Função que faz o teste t

def t_test(populacao, con_a, conv_b):

    a_dist = np.zeros(populacao)
    a_dist[:con_a] = 1
    b_dist = np.zeros(populacao)
    b_dist[:conv_b] = 1

    zscore, prob = ttest_ind(a_dist, b_dist, equal_var=False)
    
    return zscore, prob
    
    

In [81]:
z_sp, p_sp = t_test(populacao_sp, conv_a_sp, conv_b_sp)

print("Teste t SP, z-score: ", z_sp, "\t p-valor: ", p_sp)


Teste t SP, z-score:  -2.0573587072939015 	 p-valor:  0.03965461680635382


In [82]:
z_rj, p_rj  = t_test(populacao_rj, conv_a_rj, conv_b_rj)

print("Teste t SP, z-score: ", z_rj, "\t p-valor: ", p_rj)

Teste t SP, z-score:  -2.984717537008126 	 p-valor:  0.002839702990437521


#### Foi definido no início desse estudo um nível de confiança de 95%, por tanto nosso delimitador para o p-valor é de 0,05. Como em ambos os resultados o p-valor foi menor que esse delimitador podemos concluir que as políticas são estatisticamente significativas.